In [1]:
import pandas as pd
# 准备训练数据
train_data_set = pd.read_csv('train.csv')
# 获取训练集输入数据
train_data = train_data_set.iloc[0:4000,1:1601]
# 获取训练集的正确标签
train_labels = train_data_set.iloc[0:4000,[1601]]
#准备测试数据
test_data_set = pd.read_csv('test.csv')
## 可以查看一下矩阵的维度，方便处理数据
# print(test_data.shape)
test_data = test_data_set.iloc[0:3550,1:1601]
test_labels = pd.read_csv('sample_submit.csv').iloc[0:3550,[1]]
print("123")

123


In [4]:
batch_size = 50  # 一批取50个数据
lr = 0.05       # learning rate

test_data_set = pd.read_csv('test.csv')                 # shape:(3550,1601) 没有包含正确结果，和训练集不一样
test_data = test_data_set.iloc[0:1000,1:1601]           # shape:(1000,1600)
test_labels = pd.read_csv(
        'sample_submit.csv').iloc[0:1000,[1]]           # shape:(1000,1) 一一对应
print("123")

123


In [5]:
import tensorflow as tf
# 创建模型
tf_x = tf.placeholder(tf.float32,[None,40*40])   # 每批50个数据，此处暂时不管，为None
x_img = tf.reshape(tf_x,[-1,40,40,1])            # (batch, height, width, channel)
tf_y = tf.placeholder(tf.float32,[None,1])
## CNN
## 激励函数此处选择的是relu
conv1 = tf.layers.conv2d(           # shape:(40,40,1)
        inputs = x_img,
        filters = 10,               # 图像卷积后的深度
        kernel_size = 5,            # 扫描核5*5大小
        strides = 1,
        padding = 'same',
        activity_regularizer = tf.nn.relu
)                                   # shape:(40,40,10)
pool1 = tf.layers.max_pooling2d(
        inputs = conv1,
        pool_size = 2,
        strides = 2
)                                   # shape:(20,20,10)
conv2 = tf.layers.conv2d(
        inputs = pool1,
        filters = 20,
        kernel_size = 5,
        strides = 1,
        padding = 'same',
        activity_regularizer = tf.nn.relu
)                                   # shape:(20,20,20)
pool2 = tf.layers.max_pooling2d(
        inputs = conv2,
        pool_size = 2,
        strides = 2
)                                   # shape:(10,10,20)
flat_data = tf.reshape(pool2,[-1,10*10*20]) #(10*10*20, )
output = tf.layers.dense(flat_data,1) #用于全连接层，二分类0或者1。输出为某一种特定的类型。
print("123")

123


In [15]:
# loss = tf.losses.softmax_cross_entropy(tf_y,logits = output)
loss = tf.reduce_mean(-tf.reduce_sum(tf_y * tf.log(output + 1e-10),reduction_indices=[1]))
train_op = tf.train.GradientDescentOptimizer(lr).minimize(loss)

# 用于测试改模型的识别精确度
accuracy = tf.metrics.accuracy(
        labels = tf_y,
        predictions = output
)[1]
print("123")

123


In [16]:
sess = tf.Session()
# 初始化全局和本地变量
init_op = tf.group(
        tf.global_variables_initializer(), 
        tf.local_variables_initializer())
sess.run(init_op)
print("123")

123


In [17]:
# 训练，每批50个数据，一共4000个，80批一次数据集循环
for step in range(1001):                        # step = 0~999,一共1000个
    b_x = train_data.iloc[step*batch_size:(step+1)*batch_size]  # 取批数据
    b_y = train_labels.iloc[step*batch_size:(step+1)*batch_size]
    sess.run(train_op,feed_dict = {tf_x:b_x,tf_y:b_y})
    if step % batch_size ==0:
        accuracy_= sess.run(accuracy,feed_dict = {tf_x:test_data,tf_y:test_labels})
        print('Step:', step,'| test accuracy: %.8f' % accuracy_)
    
    if step == 990:
        test_data_ = test_data.iloc[23:167]
        output_= sess.run(output, feed_dict = {tf_x: test_data_})

Step: 0 | test accuracy: 0.00000000
Step: 50 | test accuracy: 0.00000000
Step: 100 | test accuracy: 0.00000000
Step: 150 | test accuracy: 0.00000000
Step: 200 | test accuracy: 0.00000000
Step: 250 | test accuracy: 0.00000000
Step: 300 | test accuracy: 0.00000000
Step: 350 | test accuracy: 0.00000000
Step: 400 | test accuracy: 0.00000000
Step: 450 | test accuracy: 0.00000000
Step: 500 | test accuracy: 0.00000000
Step: 550 | test accuracy: 0.00000000
Step: 600 | test accuracy: 0.00000000
Step: 650 | test accuracy: 0.00000000
Step: 700 | test accuracy: 0.00000000
Step: 750 | test accuracy: 0.00000000
Step: 800 | test accuracy: 0.00000000
Step: 850 | test accuracy: 0.00000000
Step: 900 | test accuracy: 0.00000000
Step: 950 | test accuracy: 0.00000000
Step: 1000 | test accuracy: 0.00000000


In [18]:
o1 = output_[0][0]
print('o1: %.8f' % o1)

o2 = output_[0]
print('o2: %.8f' % o2)

o3 = output_[1][0]
print('o3:',o3)
print('o3的类型',type(o3))

o1: -246711.90625000
o2: -246711.90625000
o3: -240683.0
o3的类型 <class 'numpy.float32'>
